## 신경망의 엔진: 그래디언트 기반 최적화

첫 번째 신경망 예제에 있는 각 층은 입력 데이터를 다음과 같이 변환한다  
output = relu(dot(W, input) * b)  
    - 이 식에서 텐서 W와 b는 층의 속성처럼 볼 수 있다 가중치(weight) 또는 훈련되는 파라미터(trainable parameter)라고 부른다(각각 커널과 편향이라고도 부르기도 함)
이런 가중치에는 학습된 정보가 담겨있다  

초기에는 가중치 행렬이 작은 난수로 채워져 있습니다(무작위 초기화 random initialization 단계) 
1. relu(dot(W, input) * b)가 처음부터 유용한 어떤 표현을 만들 것이라고 기대할 순 없다.
2. 피드백 신호에 기초하여 가중치가 점진적으로 조정된다(머신러닝의 핵심)

---
훈련은 훈련 반복 루프(traning loop) 안에서 일어난다.  
필요한 만큼 반복 루프 안에서 이런 단계가 반복된다.
---
1. 훈련 샘플 x와 이에 상응하는 타깃 y의 배치를 추출
2. x를 사용하여 네트워크를 실행하고(정방향 패스 단계),예측 y_pred를 구함
3. y_pred와 y의 차이를 측정하여 이 배치에 대한 네트워크의 손실을 계산한다
4. 배치에 대한 손실이 조금 감소되도록 네트워크의 모든 가중치를 업데이트 한다
---
결국 훈련 데이터에서 네트워크의 손실, 즉 예측 y_pred와 타깃 y의 오차가 매우 작아질 것입니다. 

개별적인 가중치 값을 업데이트 하는 방법
1. 네트워크 가중치 행렬의 원소를 모두 고정하고 관심 있는 하나만 다른 값을 적용해 보는것
    - 수천에서 수백만 개의 많은 가중치가 있기 때문에 비효율적이다(수동적)

---
신경망에 사용된 모든 연산이 미분 가능하다는 장점을 사용하여 네트워크 가중치에 대한 손실의 그래디언트를 계산 하는것이 훨씬 더 좋은 방법이다  
그래디언트의 반대방향으로 가중치를 이동하면 손실이 감소된다

---
### 2.4.1 변화율이란
- 실수 x를 새로운 실수 y로 매핑하는 연속적이고 매끄러운 함수 f(x) = y를 생각해 보자
- 이 함수가 연속적이므로 x를 조금 바꾸면 y가 조금만 변경될 것이다.
- 이 연속성의 개념을 이용해 x를 작은 값 epsilon_x만큼 증가시켰을 때 y가 epsilon_y만큼 바뀐다고 말할 수 있다

---
f(x + epsilon_x) = y + epsilon_y  
epsilon_x가 충분히 작다면 어떤 포인트 p에서 기울기 a의 선형 함수로 f를 근사할 수 있다 따라서 epsilon_y는 a * epsilon_x가 된다

---
f(x + epsilon_x) = y + a * epsilon_x

---
### 2.4.2 텐서 연산의 변화율: 그래디언트
- 그래디언트는 텐서 연산의 변화율이다
- 이는 다차원 입력, 즉 텐서를 입력으로 받는 함수에 변화율 개념을 확장시킨 것

y_pred = dot(W, x)
loss_value = loss(y_pred, y)  
입력 데이터 x와 y가 고정되어 있다면 이 함수는 W를 손실 값에 매핑하는 함수로 볼 수 있습니다.  
loss_value = f(W)


---
### 2.4.3 확률적 경사 하강법
- 변화율이 0이 되는 지점을 모두 찾고 이 중에서 어떤 포인트의 함수 값이 가장 작은지 확인하는 것
- 미니 배치 확률적 경사 하강법
1. 훈련 샘플 배치 x와 이에 상응하는 타깃 y를 추출합니다.
2. x로 네트워크를 실행하고 예측 y_pred를 구합니다.
3. 이 배치에서 y_pred와 y 사이의 오차를 측정하여 네트워크의 손실을 계산합니다.
4. 네트워크의 파라미터에 대한 손실 함수의 그래디언트를 계산합니다(역방향 패스(backward pass)).
5. 그래디언트의 반대 방향으로 파라미터를 조금 이동시킵니다. 예를 들어 W -= step * gradient처럼 하면 배치에 대한 손실이 조금 감소할 것입니다.

---
### 2.4.4변화율 연결: 역전파 알고리즘
- 3개의 텐서 연산 a, b, c와 가중치 행렬 W1, W2, W3로 구성된 네트워크 f를 예로 들어 보겠습니다.
- f(W1, W2, W3) = a(W1, b(W2, c(W3)))
- 미적분에서 이렇게 연결된 함수는 연쇄 법칙이라 부르는 항등식을 사용해 유도할 수 있다
-  f (g(x))' = f '(g(x)) * g'(x)
- 연쇄 법칙을 신경망의 그래디언트 계싼에 적용하여 역전파 알고리즘(후진 모드 자동 미분)이 탄생함
- 역전파는 최종 손실 값에서 부터 시작해 손실 값에 각 파라미터가 기여한 정도를 계산하기 위해 연쇄 법칙을 적용하여 최상위 층에서 하위 층까지 거꾸로 진행된다
---